In [ ]:
# Visualization with vnstock_ezchart 

from vnstock_ezchart import *
from vnstock import Vnstock
ezchart = MPlot() # Khởi tạo đối tượng
# Set date range
start_date = '2024-01-01'
end_date = '2025-03-19'
interval = '1D'
stock = Vnstock().stock(symbol='REE', source='VCI')
candle_df = stock.quote.history(start= start_date, end= end_date)
 
CashFlow = stock.finance.cash_flow(period='year', dropna=True)
#CashFlow.to_csv('./outputs/CashFlow.csv')
# List the columns headers 
CashFlow.columns.tolist()
# ## Transpose the CF dataframe 
#CashFlow
CashFlow_transposed = CashFlow.T
CashFlow_transposed.columns = CashFlow['yearReport']
# Drop the duplicate 'yearReport' row
CashFlow_transposed = CashFlow_transposed.drop('yearReport')
CashFlow_transposed.head()

In [ ]:
# # Port the built-in visulization method to seaborn for a more polished look

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Assuming CashFlow is a DataFrame you already have
# Normalize values to billions
data = CashFlow['Net cash inflows/outflows from operating activities'] / 1000_000_000

# Create figure with specified size
plt.figure(figsize=(12, 6))

# Sort data by year to ensure proper ordering
sorted_indices = CashFlow['yearReport'].argsort()
years = CashFlow['yearReport'].iloc[sorted_indices]
sorted_data = data.iloc[sorted_indices]

# Create the vertical bar plot with sorted data
ax = sns.barplot(
    x=years,                # Sorted years on x-axis
    y=sorted_data,          # Sorted data values on y-axis
    edgecolor='lightgrey',
    width=0.7
)

# Set title and labels with specified font sizes
plt.title('Net Operating Cashflow', fontsize=15)
plt.xlabel('Years', fontsize=10)
plt.ylabel('Value', fontsize=10)

# Turn off grid
plt.grid(False)

# Add data labels with minimal gap above bars
for i, v in enumerate(sorted_data):
    # Much smaller offset - adjust the multiplier as needed
    offset = 0.01 * max(sorted_data)
    
    ax.text(
        i,                       # x position (bar index)
        v + offset,              # Position with minimal gap
        f'{v:.2f}',              # formatted value 
        ha='center',             # horizontal alignment
        va='bottom',             # vertical alignment
        fontsize=10
    )

# Adjust y-axis limit to make room for labels
ymax = max(sorted_data) * 1.05  # Add just 5% padding to y-axis
plt.ylim(0, ymax)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Adjust layout
plt.tight_layout()

# Save the seaborn figure
plt.savefig('./outputs/Net OCF.png', dpi=300, bbox_inches='tight')

# Display the plot
plt.show()

In [ ]:
Ratio = stock.finance.ratio(period='year', lang='vi', dropna=True)
#Ratio.to_csv('./outputs/ratios.csv')
Ratio.columns.to_list()
#Transpose the data frame to display on the web.
Ratio_transposed = Ratio.T
Ratio_transposed.columns=Ratio_transposed.iloc[1]
Ratio_transposed = Ratio_transposed.iloc[3:]

In [ ]:
import matplotlib.pyplot as plt
Ratio_plot = Ratio.copy()
# Select the two columns
col1 = ('Chỉ tiêu khả năng sinh lợi', 'Biên lợi nhuận gộp (%)')
col2 = ('Chỉ tiêu cơ cấu nguồn vốn', 'Nợ/VCSH')

# Plot
plt.figure(figsize=(10, 6))
plt.plot(Ratio_plot.index, Ratio_plot[col1], marker='o', label='Biên lợi nhuận gộp (%)')
plt.plot(Ratio_plot.index, Ratio_plot[col2], marker='s', label='Nợ/VCSH')
plt.xlabel('Năm')
plt.ylabel('Giá trị (%)')
plt.title('So sánh Biên lợi nhuận gộp và Nợ/VCSH theo năm')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Select the columns
col_roe = ('Chỉ tiêu khả năng sinh lợi', 'ROE (%)')
col_debt_equity = ('Chỉ tiêu cơ cấu nguồn vốn', 'Nợ/VCSH')

# Prepare the DataFrame for plotting
df_plot = Ratio[[col_roe, col_debt_equity]].copy()
df_plot.columns = ['ROE (%)', 'Nợ/VCSH']

# Optional: add company or year info for further analysis
if ('Meta', 'CP') in Ratio.columns:
    df_plot['CP'] = Ratio[('Meta', 'CP')]
if ('Meta', 'Năm') in Ratio.columns:
    df_plot['Năm'] = Ratio[('Meta', 'Năm')]

# Scatter plot with regression line
plt.figure(figsize=(8, 6))
sns.regplot(data=df_plot, x='Nợ/VCSH', y='ROE (%)', scatter_kws={'alpha':0.7})
plt.title('Mối quan hệ giữa Đòn bẩy tài chính (Nợ/VCSH) và ROE (%)')
plt.xlabel('Nợ/VCSH')
plt.ylabel('ROE (%)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define the five most meaningful metrics (six columns for all pairwise relationships)
selected_cols = [
    ('Chỉ tiêu khả năng sinh lợi', 'ROE (%)'),
    ('Chỉ tiêu cơ cấu nguồn vốn', 'Nợ/VCSH'),
    ('Chỉ tiêu hiệu quả hoạt động', 'Vòng quay tài sản'),
    ('Chỉ tiêu khả năng sinh lợi', 'Biên lợi nhuận ròng (%)'),
    ('Chỉ tiêu thanh khoản', 'Chỉ số thanh toán hiện thời'),
    ('Chỉ tiêu định giá', 'P/S'),
]

# Subset the DataFrame
df_pair = Ratio[selected_cols].copy()

# Use only the second part of each column tuple (the metric name)
df_pair.columns = [col[1] for col in df_pair.columns]

# Optional: Remove rows with missing values for these columns
#df_pair = df_pair.dropna()

# Create the pairplot
sns.pairplot(df_pair, diag_kind='kde', corner=True)
plt.suptitle('Pairplot of Key Financial Metrics', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import numpy as np
# import pandas as pd

# # Define the five most meaningful metrics (six columns for all pairwise relationships)
# selected_cols = [
#     ('Chỉ tiêu khả năng sinh lợi', 'ROE (%)'),
#     ('Chỉ tiêu cơ cấu nguồn vốn', 'Nợ/VCSH'),
#     ('Chỉ tiêu hiệu quả hoạt động', 'Vòng quay tài sản'),
#     ('Chỉ tiêu khả năng sinh lợi', 'Biên lợi nhuận ròng (%)'),
#     ('Chỉ tiêu thanh khoản', 'Chỉ số thanh toán hiện thời'),
#     ('Chỉ tiêu định giá', 'P/S'),
# ]

# # Subset the DataFrame
# df_pair = Ratio[selected_cols].copy()

# # Use only the second part of each column tuple (the metric name)
# df_pair.columns = [col[1] for col in df_pair.columns]

# # Optional: Remove rows with missing values for these columns
# #df_pair = df_pair.dropna()

# # Create a plotly figure using px.scatter_matrix for the pairplot
# fig = px.scatter_matrix(
#     df_pair,
#     dimensions=df_pair.columns,
#     title="Pairplot of Key Financial Metrics",
#     labels={col: col for col in df_pair.columns},  # Use original column names as labels
#     color_discrete_sequence=['blue'],
#     opacity=0.6
# )

# # Update layout for better appearance
# fig.update_layout(
#     title={
#         'y':0.95,
#         'x':0.5,
#         'xanchor': 'center',
#         'yanchor': 'top'
#     },
#     dragmode='select',
#     width=1000,
#     height=900,
# )

# # Update traces for diagonal plots to show distributions
# for i, col in enumerate(df_pair.columns):
#     fig.update_traces(
#         diagonal_visible=True, 
#         showupperhalf=False,  # Only show lower half (like corner=True in seaborn)
#         selector=dict(dimensions=[col])
#     )

# # Update axes to include zero when appropriate
# fig.update_xaxes(showline=True, linewidth=1, linecolor='gray', mirror=True)
# fig.update_yaxes(showline=True, linewidth=1, linecolor='gray', mirror=True)

# # Show the plot
# fig.show()

# # Save to HTML for sharing
# fig.write_html("./outputs/financial_metrics_pairplot.html")
# print("Interactive pairplot saved to financial_metrics_pairplot.html")

In [ ]:
# Transform the balance sheet from long format to wide format. 

def BS_wide(stock=None):
    """
    Transform balance sheet data into a wide format with years as columns and metrics as rows.
    
    Parameters:
    -----------
    stock : object, default=None
        The stock ticker object containing financial data.
        If None, will create a default stock object for REE from VCI source.
    
    Returns:
    --------
    pandas.DataFrame
        Transformed balance sheet with years as columns and financial metrics as rows
    """
    
    # Create default stock object if not provided
    if stock is None:
        stock = Vnstock().stock(symbol='REE', source='VCI')
    
    # Get the balance sheet data
    BS = stock.finance.balance_sheet(period='year', lang='en', dropna=True)
    
    # Transpose the DataFrame
    BS_wide = BS.T
    
    # Promote header by setting column names using the second row (index 1)
    BS_wide.columns = BS_wide.iloc[1]
    
    # Keep only the data rows (skip the first 3 rows)
    BS_wide = BS_wide.iloc[3:]
    
    return BS_wide
# BS_wide(stock)

In [ ]:

BalanceSheet = stock.finance.balance_sheet(period='year', lang='en', dropna=True)
BalanceSheet_Transposed = BalanceSheet.T
BalanceSheet_Transposed.columns = BalanceSheet_Transposed.iloc[1]
BalanceSheet_Transposed = BalanceSheet_Transposed.iloc[3:]
BalanceSheet_Transposed.head()


#Transform Income statement from long format to wide format. 

IncomeStatement = stock.finance.income_statement(period='year', lang='en', dropna=True)
IncomeStatement_Transpose= IncomeStatement.T
IncomeStatement_Transpose.columns = IncomeStatement_Transpose.iloc[1]
IncomeStatement_Transpose = IncomeStatement_Transpose.iloc[3:]
IncomeStatement_Transpose.head()


BalanceSheet_Transposed.to_csv('./outputs/REE_BalanceSheet_Transposed.csv')
IncomeStatement_Transpose.to_csv('./outputs/REE_IncomeStatement_Transpose.csv')
CashFlow_transposed.to_csv('./outputs/REE_CashFlow_transposed.csv')


# In[62]:


# import os
# import pandas as pd

# def save_financial_statements_to_csv(balance_sheet_df, income_statement_df, cashflow_df, output_dir='./outputs'):
#     """
#     Save financial statements DataFrames to CSV files in the specified output directory.
    
#     Parameters:
#     -----------
#     balance_sheet_df : pandas.DataFrame
#         Balance Sheet DataFrame
#     income_statement_df : pandas.DataFrame
#         Income Statement DataFrame
#     cashflow_df : pandas.DataFrame
#         Cash Flow Statement DataFrame
#     output_dir : str
#         Directory path where CSV files will be saved (default: './outputs')
#     """
    
#     # Create output directory if it doesn't exist
#     try:
#         os.makedirs(output_dir, exist_ok=True)
#         print(f"Output directory '{output_dir}' is ready.")
#     except Exception as e:
#         print(f"Error creating directory: {e}")
#         return
    
#     # Dictionary of DataFrames and their corresponding filenames
#     statements = {
#         'REE_BalanceSheet_Transposed.csv': balance_sheet_df,
#         'REE_IncomeStatement_Transpose.csv': income_statement_df,
#         'REE_CashFlow_transposed.csv': cashflow_df
#     }
    
#     # Save each DataFrame to CSV
#     for filename, df in statements.items():
#         try:
#             file_path = os.path.join(output_dir, filename)
#             df.to_csv(file_path)
#             print(f"Successfully saved {filename}")
#         except Exception as e:
#             print(f"Error saving {filename}: {e}")

# # Example usage:
# # save_financial_statements_to_csv(
# #     BalanceSheet_Transposed,
# #     IncomeStatement_Transpose,
# #     CashFlow_transposed
# # )